In [1]:
import cv2

cap = cv2.VideoCapture(0)
# If using a video file instead, replace 0 with the file path like: cap = cv2.VideoCapture("filename.mp4")

# Set up initial frame
ret, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
prev_gray = cv2.GaussianBlur(prev_gray, (5, 5), 0)

# Define motion detection parameters
motion_threshold = 30
min_area = 500

while True:
    ret, frame2 = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale and blur it
    current_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    current_gray = cv2.GaussianBlur(current_gray, (5, 5), 0)

    # Compute difference between current frame and initial frame
    frame_diff = cv2.absdiff(prev_gray, current_gray)
    thresholded = cv2.threshold(frame_diff, motion_threshold, 255, cv2.THRESH_BINARY)[1]

    # Dilate the thresholded image to fill in holes
    dilated = cv2.dilate(thresholded, None, iterations=2)

    # Find contours and draw bounding rectangles around moving objects
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show resulting frames
    cv2.imshow('Original', frame2)

    # Update previous frame and handle exit condition
    prev_gray = current_gray
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Clean up resources
cap.release()
cv2.destroyAllWindows()